# How to use Flux

Flux is a library for machine learning geared towards high-performance production pipelines. It comes "batteries-included" with many useful tools built in, but also lets you use the full power of the Julia language where you need it. We follow a few key principles:

- Doing the obvious thing. Flux has relatively few explicit APIs for features like regularisation or embeddings. Instead, writing down the mathematical form will work – and be fast.
- Extensible by default. Flux is written to be highly extensible and flexible while being performant. Extending Flux is as simple as using your own code as part of the model you want - it is all high level Julia code. When in doubt, it’s well worth looking at the source. If you need something different, you can easily roll your own.
- Performance is key. Flux integrates with high-performance AD tools such as Zygote.jl for generating fast code. Flux optimizes both CPU and GPU performance. Scaling workloads easily to multiple GPUs can be done with the help of Julia's GPU tooling and projects like DaggerFlux.jl.
- Play nicely with others. Flux works well with Julia libraries from data frames and images to differential equation solvers, so you can easily build complex data processing pipelines that integrate Flux models.

In [ ]:
using Flux, PlotlyJS, Random

In [ ]:
Random.seed!(1234);

### GPU

In [ ]:
using CUDA
# CUDA.functional()

In [ ]:
dense_layer = Dense(10, 5);
tensor = rand(10);

In [ ]:
results_path = "results";
!isdir(results_path) && mkdir(results_path);

## Input Data

### Generate random data

The target `y` represents two classes generated by two circular distribution that are not linearly separable because class 0 surrounds class 1.

In [ ]:
# dataset params
N = 50000;
factor = 0.1;
noise = 0.1;

In [ ]:
# Adapted From SyntheticDatasets
using DataFrames
using PyCall
sk = pyimport("sklearn.datasets")

function convert(features::Array{T, 2}, labels::Array{D, 1})::DataFrame where {T <: Number, D <: Number}
    df = DataFrame()

    for i = 1:size(features)[2]
        df[!, Symbol("feature_$(i)")] = eltype(features)[]
    end
    
    df[!, :label] = eltype(labels)[]
    
    for label in unique(labels)
        for i in findall(r->r == label, labels)
            push!(df, (features[i, :]... , label))
        end
    end

    return df
end

function convert(features::Array{T, 2}, labels::Array{D, 2})::DataFrame where {T <: Number, D <: Number}
    df = DataFrame()

    for i = 1:size(features)[2]
        df[!, Symbol("feature_$(i)")] = eltype(features)[]
    end

    for i = 1:size(labels)[2]
        df[!, Symbol("label_$(i)")] = eltype(labels)[]
    end
    
    for row in 1:size(features)[1]
        push!(df, (features[row, :]... , labels[row, :]...))
    end
    
    return df
end


(features, labels) = sk.make_circles(
    n_samples=N,
    shuffle=true,
    noise=noise,
    factor=factor
);

circles = convert(features, labels);

In [ ]:
labels_column_name = "label"

y = circles[:, [labels_column_name]];
y = Matrix(y);

X = select!(circles, Not(labels_column_name));
X = Matrix(X);


In [ ]:
# define outcome matrix
Y = zeros(Float64, (N, 2));
for c ∈ [0, 1]
    # mask = collect(Iterators.flatten(y .== c))
    mask = vec(y .== c)
    Y[mask, c+1] .= 1.0
end

In [ ]:
"Shape of: X: $(size(X)) | Y: $(size(Y)) | y: $(size(y))"

### Visualize Data

In [ ]:
using PlotlyJS
using DataFrames

##### Set PlotlyJS Theme

In [ ]:
templates.default = "plotly_dark";
PlotlyJS.templates

In [ ]:
df = DataFrame(
    X1=X[:, 1],
    X2=X[:, 2],
    Label=y[:, 1];
);

# ax = PlotlyJS.plot(
#     df,
#     x=:X1,
#     y=:X2,
#     m=1,
#     color=:Label,
#     kind="scatter",
#     mode="markers",
#     labels=Dict(
#         :X1 => "X",
#         :X2 => "Y",
#         :Label => "Labels"
#     ),
#     marker=attr(size=8, line=attr(width=1, color="DarkSlateGrey")),
#     PlotlyJS.Layout(
#         title="Circles Dataset Visualization",
#         width=600, height=600,
#     )
# )


## Build Flux Model

[[FLUX_TUTORIAL_MODEL_BUILDING]]

### Define Architecture

In [ ]:
model = Chain(
    Dense(2, 3, Flux.σ),
    Dense(3, 2),
    softmax
);

Flux.loadparams!(model, map(p -> p .= randn.(), Flux.params(model)))

1 - FLUX_TUTORIAL_MODEL_ARCHITECTURE

2 - FLUX_TUTORIAL_MODEL_ARCHITECTURE


In [ ]:
model

### ML Flow Experiment Tracking

MLflow is a platform to streamline machine learning development, including tracking experiments, packaging code into reproducible runs, and sharing and deploying models. MLflow offers a set of lightweight APIs that can be used with any existing machine learning application or library (TensorFlow, PyTorch, XGBoost, etc), wherever you currently run ML code (e.g. in notebooks, standalone applications or the cloud)

In [ ]:
using PyCall

mlflow = pyimport("mlflow")

MLF_EXPERIMENT_NAME = "How To Use Flux Julia"
MLF_EXPERIMENT_ID = 0

try
    MLF_EXPERIMENT_ID = mlflow.get_experiment_by_name(MLF_EXPERIMENT_NAME).experiment_id
catch e
    MLF_EXPERIMENT_ID = mlflow.create_experiment(MLF_EXPERIMENT_NAME)
end

mlflow.set_experiment(experiment_id=MLF_EXPERIMENT_ID)


In [ ]:
using Flux, MLUtils, MLBase
function performance_evaluation_dict(
    ŷ,
    y,
    phase,
)
    ŷ, y = vec(ŷ), vec(y);

    error_rate = MLBase.errorrate(ŷ, y);

    accuracy = MLBase.correctrate(ŷ, y);

    loss = Flux.binarycrossentropy(ŷ, y);

    roc_nums::ROCNums = MLBase.roc(ŷ, y);

    fpr = MLBase.false_positive_rate(roc_nums);
    fnr = MLBase.false_negative_rate(roc_nums);

    f1_score = MLBase.f1score(roc_nums);

    metrics_dict = Dict(
        "$phase error_rate" => error_rate,
        "$phase accuracy" => accuracy,
        "$phase loss" => loss,
        "$phase fpr" => fpr,
        "$phase fnr" => fnr,
        "$phase f1_score" => f1_score,
    );

    metrics_dict
end;


In [ ]:
predict(x) = map(x -> x[1] - 1, argmax(model(x), dims=1));
epochs = 10
optimiser = Flux.RMSProp();

### Custom Train Validation Loops

In [ ]:
using MLBase, MLUtils, Flux

function train_loop(train_data, model, optimiser, batch_size)
    Flux.trainmode!(model)

    for (x, y) ∈ eachobs(train_data, batchsize=batch_size)
        # ... train supervised model on minibatches here
        grads = gradient(Flux.params(model)) do
            training_loss = Flux.binarycrossentropy(model(x), y)

            # Code inserted here will be differentiated, unless you need that gradient information
            # it is better to do the work outside this block.

            return training_loss
        end

        # Insert whatever code you want here that needs training_loss, e.g. logging.
        # logging_callback(training_loss)
        # Insert what ever code you want here that needs gradient.
        # E.g. logging with TensorBoardLogger.jl as histogram so you can see if it is becoming huge.

        Flux.update!(optimiser, Flux.params(model), grads)

        # Here you might like to check validation set accuracy, and break out to do early stopping.
    end

    X, Y = train_data

    Y = map(x -> x[1] - 1, argmax(Y, dims=1))
    Ŷ = predict(X)

    Y, Ŷ
end;

function test_loop(test_data, model, optimiser, batch_size)
    Flux.testmode!(model)

    for (x, y) ∈ eachobs(test_data, batchsize=batch_size)
        # # ... train supervised model on minibatches here
        # grads = gradient(Flux.params(model)) do
        #     training_loss = Flux.binarycrossentropy(model(x), y)

        #     # Code inserted here will be differentiated, unless you need that gradient information
        #     # it is better to do the work outside this block.

        #     return training_loss
        # end

        # # Insert whatever code you want here that needs training_loss, e.g. logging.
        # # logging_callback(training_loss)
        # # Insert what ever code you want here that needs gradient.
        # # E.g. logging with TensorBoardLogger.jl as histogram so you can see if it is becoming huge.

        # Flux.update!(optimiser, Flux.params(model), grads)

        # # Here you might like to check validation set accuracy, and break out to do early stopping.
    end

    X, Y = test_data

    Y = map(x -> x[1] - 1, argmax(Y, dims=1))
    Ŷ = predict(X)

    Y, Ŷ
end;


X_ = X';
Y_ = Y';

# # shuffle observations
Xs, Ys = shuffleobs((X_, Y_))

# We leave out 15 % of the data for testing
cv_data, test_data = splitobs((Xs, Ys); at=0.85)

print(size(Y_))

batch_size = 128

# # Next we partition the data using a 10-fold scheme.
for (k, (train_data, val_data)) ∈ enumerate(kfolds(cv_data; k=5))
    mlflow.start_run(run_name="Fold $(k)");

    for epoch = 1:20

        train_Y, train_Ŷ = train_loop(train_data, model, optimiser, batch_size)
        val_Y, val_Ŷ = test_loop(val_data, model, optimiser, batch_size)

        train_metrics_dict = performance_evaluation_dict(
            train_Ŷ,
            train_Y,
            "train",
        )

        val_metrics_dict = performance_evaluation_dict(
            val_Ŷ,
            val_Y,
            "validation",
        )

        # mlflow.log_params(hyperparameters_configs.to_dict())
        mlflow.log_metrics(train_metrics_dict, step=epoch);
        mlflow.log_metrics(val_metrics_dict, step=epoch);
    end

    mlflow.end_run()
end

### Train & Test Accuracy & Loss Per Epoch

Run command `mlflow server -p 5001` to observe experiment tracking's results.

## Get Weights

In [ ]:
model_params = Flux.params(model);
[size(p) for p in model_params]

## Plot Decision Boundary

The visualization of the decision boundary resembles the result from the manual network implementation. The training with **Flux** runs a multiple faster, though.

In [ ]:
function meshgrid(x, y)
    mg = Iterators.product(x, y)

    xx = first.(mg)
    yy = last.(mg)

    xx, yy
end;

In [ ]:
# 500 instead of 200 for more resulotion boundry in the coutour plot.
n_vals = 500;
x1 = range(-1.5, 1.5, n_vals);
x2 = range(-1.5, 1.5, n_vals);
# create the grid
xx, yy = meshgrid(x1, x2);

In [ ]:
# Initialize and fill the feature space
feature_space = zeros(n_vals, n_vals);
f = false;
for i ∈ 1:n_vals
    for j ∈ 1:n_vals
        X_ = [xx[i, j]; yy[i, j]];
        result = predict(X_);

        # if (result != [0])
        #     print(result);
        #     f = true;
        #     break;
        # end

        # index = argmax(probabilities);
        feature_space[i, j] = result[1];
    end
    # if (f)
    #     break;
    # end
end

In [ ]:
df = DataFrames.transform(df, :Label => ByRow(label -> label == 0 ? "Outer Circle" : "Inner Circle") => :State);

alpha = 0.15;
colorscale = [[0, "rgba(0,0,255,$alpha)"], [1, "rgba(255,255,0,$alpha)"]]

PlotlyJS.plot([
        contour(
            x=x1, # horizontal axis
            y=x2, # vertical axis
            z=feature_space', # data

            # contours_coloring="lines",

            # autocolorscale=true,
            colorscale=colorscale,

            line_width=1,
            line_smoothing=0.85,
            colorbar=attr(
                title="Decision Boundry", # title here
                titleside="right",
                titlefont=attr(
                    size=14,
                    family="Arial, sans-serif"),
                thickness=25,
                thicknessmode="pixels",
                len=0.8,
                lenmode="fraction",
                outlinewidth=0,
            ),

            # Smooth Coloring based on Z (In this case Z = 0, 1)
            contours=attr(
                coloring ="heatmap",
                # showlabels = true, # show labels on contours
                labelfont = attr( # label font properties
                    size = 12,
                    color = "white",
                )
            )
        ),
        
        PlotlyJS.scatter(
            df,
            x=:X1,
            y=:X2,

            # NOTE: marker_color for scatter() and color for plot()
            marker_color=:Label,
            
            text=:State,
            
            mode="markers",
            labels=Dict(
                :X1 => "X",
                :X2 => "Y",
                :Label => "Labels"
            ),
            marker=attr(size=8, line=attr(width=1, color="DarkSlateGrey")),
        ),
        ],
    PlotlyJS.Layout(
        title="Circles Dataset Visualization",
        width=700, height=700,
    )
)